References:
1. https://github.com/jf20541/LogisticRegressionPyTorch/blob/main/src/pytorchmodel.py
2. https://pytorch.org/tutorials/beginner/basics/

TODOs

1. Add regularization
2. Explore LBFGS in PyTorch
3. Implement fairness variant

In [1]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

random_state = 1
torch.manual_seed(random_state)

/opt/homebrew/Caskroom/miniforge/base/envs/bnn/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('../data/cleaned.csv', index_col=0)
df.head()

,Duration,Credit Amount,Installment rate,Residence,Age,Number of credits,Maintenance,Target,Account Status_<0,Account Status_<200,...,Housing_own,Housing_rent,Job_management/ highly qualified employee,Job_skilled employee / official,Job_unemployed/ unskilled - non-resident,Job_unskilled - resident,Telephone_none,Telephone_yes,Foreign_no,Foreign_yes
0,6,1169,4,4,67,2,1,1,1,0,...,1,0,0,1,0,0,0,1,0,1
1,48,5951,2,2,22,1,1,0,0,1,...,1,0,0,1,0,0,1,0,0,1
2,12,2096,2,3,49,1,2,1,0,0,...,1,0,0,0,0,1,1,0,0,1
3,42,7882,2,4,45,1,2,1,1,0,...,0,0,0,1,0,0,1,0,0,1
4,24,4870,3,4,53,2,2,0,1,0,...,0,0,0,1,0,0,1,0,0,1


In [3]:
Xs = torch.tensor(df.loc[:, df.columns != 'Target'].values)
ys = torch.tensor(df['Target'].values)
ys = ys.reshape(ys.shape[0], 1)

# class CustomDataset(Dataset):
#     def __init__(self, Xs, ys):
#         self.Xs = torch.tensor(Xs.values)
#         temp = torch.tensor(ys.values)
#         self.ys = temp.reshape(temp.shape[0], 1)

#     def __len__(self):
#         return len(self.Xs)

#     def __getitem__(self, idx):
#         return self.Xs[idx], self.ys[idx]

In [4]:
class LogisticRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return torch.sigmoid(self.linear(x))

In [5]:
model = LogisticRegression(Xs.shape[1], 1)
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

# all_data = CustomDataset(Xs, ys)

# train_dataloader = DataLoader(all_data, batch_size=128, shuffle=True)
# test_dataloader = DataLoader(all_data, batch_size=128, shuffle=True)

In [6]:
def train_loop(Xs, ys, model, loss_fn, optimizer):
    size = Xs.shape[0]
    # Compute prediction and loss
    pred = model(Xs.float())
    loss = loss_fn(pred, ys.float())

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    loss = loss.item()
    print(f"loss: {loss:>7f}")


def test_loop(Xs, ys, model, loss_fn):
    size = Xs.shape[0]
    test_loss, correct = 0, 0

    with torch.no_grad():
        pred = model(Xs.float())
        test_loss += loss_fn(pred, ys.float()).item()
        correct += ((pred > 0.5).float() == ys).type(torch.float).sum().item()
        
    correct /= size

    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [7]:
epochs = 20
for t in range(epochs):
#     print(f"Epoch {t+1}\n-------------------------------")
    train_loop(Xs, ys, model, loss_fn, optimizer)
    test_loop(Xs, ys, model, loss_fn)
print("Done!")

loss: 62.105740
Test Error: 
 Accuracy: 30.0%, Avg loss: 58.332043 

loss: 58.332043
Test Error: 
 Accuracy: 30.0%, Avg loss: 52.791943 

loss: 52.791943
Test Error: 
 Accuracy: 30.0%, Avg loss: 45.582142 

loss: 45.582142
Test Error: 
 Accuracy: 30.0%, Avg loss: 35.924740 

loss: 35.924740
Test Error: 
 Accuracy: 30.0%, Avg loss: 24.857281 

loss: 24.857281
Test Error: 
 Accuracy: 29.9%, Avg loss: 10.310245 

loss: 10.310245
Test Error: 
 Accuracy: 69.9%, Avg loss: 6.651823 

loss: 6.651823
Test Error: 
 Accuracy: 70.0%, Avg loss: 16.610487 

loss: 16.610487
Test Error: 
 Accuracy: 70.0%, Avg loss: 21.295042 

loss: 21.295042
Test Error: 
 Accuracy: 70.0%, Avg loss: 23.176250 

loss: 23.176250
Test Error: 
 Accuracy: 70.0%, Avg loss: 25.534040 

loss: 25.534040
Test Error: 
 Accuracy: 70.0%, Avg loss: 26.422312 

loss: 26.422312
Test Error: 
 Accuracy: 70.0%, Avg loss: 27.039875 

loss: 27.039875
Test Error: 
 Accuracy: 70.0%, Avg loss: 27.479313 

loss: 27.479313
Test Error: 
 Accura